In [111]:
import os
import tvm
from tvm.script import ir as I
from tvm.script import tir as T
from tvm import autotvm, auto_scheduler
from tvm.autotvm.tuner import XGBTuner, GATuner, RandomTuner, GridSearchTuner
from tvm import meta_schedule as ms
from tvm.ir import IRModule
from tvm import relax
from tvm import rpc
from tvm.contrib import utils, ndk
x_shape = 4096
w_w_x = 512
w_s_x = 128
w_y = 11008*2
func_name = "main"
@I.ir_module
class ModuleSrc:
    @T.prim_func(private=False)
    def main(lv1611: T.Buffer((T.int64(512), T.int64(w_y)), "uint32"),
             lv1612: T.Buffer((T.int64(128), T.int64(w_y)), "float16"),
             lv1622: T.Buffer((T.int64(1), T.int64(1), T.int64(4096)), "float16"),
             p_output0_intermediate: T.Buffer((T.int64(1), T.int64(1), T.int64(w_y)), "float16")):
        T.func_attr({"global_symbol": "main", "tir.noalias": T.bool(True)})
        # with T.block("root"):
        var_decode_intermediate = T.alloc_buffer((T.int64(4096), T.int64(w_y)), "float16")
        var_matmul_intermediate = T.alloc_buffer(
            (T.int64(1), T.int64(1), T.int64(w_y)), "float16"
        )
        for i, j in T.grid(T.int64(4096), T.int64(w_y)):
            with T.block("decode"):
                v_i, v_j = T.axis.remap("SS", [i, j])
                T.reads(lv1611[v_i // T.int64(8), v_j], lv1612[v_i // T.int64(32), v_j])
                T.writes(var_decode_intermediate[v_i, v_j])
                var_decode_intermediate[v_i, v_j] = (
                    T.Cast(
                        "float16",
                        T.bitwise_and(
                            T.shift_right(
                                lv1611[v_i // T.int64(8), v_j],
                                T.Cast("uint32", v_i % T.int64(8)) * T.uint32(4),
                            ),
                            T.uint32(15),
                        ),
                    )
                    - T.float16(7)
                ) * lv1612[v_i // T.int64(32), v_j]
        for i0, i1, i2, k in T.grid(T.int64(1), T.int64(1), T.int64(w_y), T.int64(4096)):
            with T.block("matmul"):
                v_i0, v_i1, v_i2, v_k = T.axis.remap("SSSR", [i0, i1, i2, k])
                T.reads(lv1622[v_i0, v_i1, v_k], var_decode_intermediate[v_k, v_i2])
                T.writes(var_matmul_intermediate[v_i0, v_i1, v_i2])
                with T.init():
                    var_matmul_intermediate[v_i0, v_i1, v_i2] = T.float16(0)
                var_matmul_intermediate[v_i0, v_i1, v_i2] = (
                    var_matmul_intermediate[v_i0, v_i1, v_i2]
                    + lv1622[v_i0, v_i1, v_k] * var_decode_intermediate[v_k, v_i2]
                )
        for ax0, ax1, ax2 in T.grid(T.int64(1), T.int64(1), T.int64(w_y)):
            with T.block("T_update"):
                v_ax0, v_ax1, v_ax2 = T.axis.remap("SSS", [ax0, ax1, ax2])
                T.reads(var_matmul_intermediate[v_ax0, v_ax1, v_ax2])
                T.writes(p_output0_intermediate[v_ax0, v_ax1, v_ax2])
                p_output0_intermediate[v_ax0, v_ax1, v_ax2] = var_matmul_intermediate[v_ax0, v_ax1, v_ax2]

@I.ir_module
class ModuleToManual:
    @T.prim_func(private=False)
    def main(lv1611: T.Buffer((T.int64(512), T.int64(w_y)), "uint32"),
             lv1612: T.Buffer((T.int64(128), T.int64(w_y)), "float16"),
             lv1622: T.Buffer((T.int64(1), T.int64(1), T.int64(4096)), "float16"),
             p_output0_intermediate: T.Buffer((T.int64(1), T.int64(1), T.int64(w_y)), "float16")):
        T.func_attr({"global_symbol": "main", "tir.noalias": T.bool(True)})
        # with T.block("root"):
        var_decode_intermediate = T.alloc_buffer((T.int64(4096), T.int64(w_y)), "float16")
        var_matmul_intermediate = T.alloc_buffer(
            (T.int64(1), T.int64(1), T.int64(w_y)), "float16"
        )
        for i, j in T.grid(T.int64(4096), T.int64(w_y)):
            with T.block("decode"):
                v_i, v_j = T.axis.remap("SS", [i, j])
                T.reads(lv1611[v_i // T.int64(8), v_j], lv1612[v_i // T.int64(32), v_j])
                T.writes(var_decode_intermediate[v_i, v_j])
                var_decode_intermediate[v_i, v_j] = (
                    T.Cast(
                        "float16",
                        T.bitwise_and(
                            T.shift_right(
                                lv1611[v_i // T.int64(8), v_j],
                                T.Cast("uint32", v_i % T.int64(8)) * T.uint32(4),
                            ),
                            T.uint32(15),
                        ),
                    )
                    - T.float16(7)
                ) * lv1612[v_i // T.int64(32), v_j]
        for i0, i1, i2, k in T.grid(T.int64(1), T.int64(1), T.int64(w_y), T.int64(4096)):
            with T.block("matmul"):
                v_i0, v_i1, v_i2, v_k = T.axis.remap("SSSR", [i0, i1, i2, k])
                T.reads(lv1622[v_i0, v_i1, v_k], var_decode_intermediate[v_k, v_i2])
                T.writes(var_matmul_intermediate[v_i0, v_i1, v_i2])
                with T.init():
                    var_matmul_intermediate[v_i0, v_i1, v_i2] = T.float16(0)
                var_matmul_intermediate[v_i0, v_i1, v_i2] = (
                    var_matmul_intermediate[v_i0, v_i1, v_i2]
                    + lv1622[v_i0, v_i1, v_k] * var_decode_intermediate[v_k, v_i2]
                )
        for ax0, ax1, ax2 in T.grid(T.int64(1), T.int64(1), T.int64(w_y)):
            with T.block("T_update"):
                v_ax0, v_ax1, v_ax2 = T.axis.remap("SSS", [ax0, ax1, ax2])
                T.reads(var_matmul_intermediate[v_ax0, v_ax1, v_ax2])
                T.writes(p_output0_intermediate[v_ax0, v_ax1, v_ax2])
                p_output0_intermediate[v_ax0, v_ax1, v_ax2] = var_matmul_intermediate[v_ax0, v_ax1, v_ax2]

In [112]:
## ref to mlc-llm/dispatch/dispatch_tir_operator_adreno.py
def sch_fused_decode5_fused_matmul6_silu1(func):
    """
    Summarize
    --------------------------------
    gemv整体分析优化内容:
    1. 内存方面
        
        1.1 将参与多次计算的 `v` 搬运到shared_memory中
        
        1.2 将参与单次运算, 且较小的 quantized_weight_scale搬运到local memory中, 这一步感觉作用不大，可以做实验看下去掉的效果

        1.3 通过tiling, 使计算满足cache line, 提高cache命中
    
    2. 计算方面
        
        2.1 计算合并(内联), 如将dequantize操作与gemv合并, 将中间结果从local_memory -> register, 降低中间缓存及其内存搬运导致的耗时, 大幅度提高计算效率

        2.2 通过split和reorder操作实现tiling, 提高cache命中, 减少内存搬运

        2.3 绑定硬件线程(block & thread)

        2.4 使用vectorize, 利用SIMD指令集加速 load/store & ALU计算

        2.5 实际上还将内存搬运(global memory -> local/shared memory)与计算独立开来，提高内存搬运效率(需要看生成的源码如何实现)

        2.6 `v` 的shared memory搬移时, 使用cooperative_fetch, 暂时没搞明白如何实现, 需要生成的cuda代码

    3. 改进点?
        没有看到用unroll操作, 其余的好像都用到了 -- 用了 vectorize, 不需要unroll了
        
    opencl source
    ----------------------------------------------------------------
    // Function: main_kernel
    #ifdef cl_khr_fp16
    #pragma OPENCL EXTENSION cl_khr_fp16 : enable
    #elif defined(cl_amd_fp16)
    #pragma OPENCL EXTENSION cl_amd_fp16 : enable
    #else
    #error "Half precision floating point not supported by OpenCL implementation on your device." 
    #endif
    // lv1611 - w_int4
    // lv1612 - scale (group size = 32)
    // lv1622 - input x
    // kernel:  输入 [512, 11008], [128, 11008], [1,1,4096] -> [1,1 11008]
    //          工作组设置: blockIdx = 43, threadIdx = 64
    //          计算: output[4] = input[4096] x 4列W [1,1,4096] => [4]
    __kernel void main_kernel(__global uint* restrict lv1611, __global half* restrict lv1612, __global half* restrict lv1622, __global half* restrict p_output0_intermediate) {
    // 这里标注的是local内存, 即
    __local half lv1622_shared[4096];
    // 以下三个half4都是private memory, 如果resgiter足够的话, 会放到resgiter中, 如果不够会放到 local memory -> global memory
    half4 var_matmul_intermediate_local[1];
    half4 lv1612_local[1];
    uint4 lv1611_local[1];
    ## local_id范围[0,63], 64*8*8=4096, 即一个workgroup把 [1,1,4096]的输入都读到了shared_memory中
    for (int ax2_0 = 0; ax2_0 < 8; ++ax2_0) {
        vstore8(vload8(0, lv1622 + ((ax2_0 * 512) + ((convert_int(get_local_id(0))) * 8))), 0, lv1622_shared + ((ax2_0 * 512) + ((convert_int(get_local_id(0))) * 8)));
    }
    // 中间缓存初始化
    var_matmul_intermediate_local[0] = ((half4)((half)0.000000e+00f, (half)0.000000e+00f, (half)0.000000e+00f, (half)0.000000e+00f));
    barrier(CLK_LOCAL_MEM_FENCE);
    # W的对应关系, 按行存储: lv1622_shared[v_i0, v_i1, v_k], lv1611_local[v_k // T.int64(8), v_i2], lv1612_local[v_k // T.int64(32), v_i2]
    for (int k_0 = 0; k_0 < 128; ++k_0) {
        // load scale(group size = 32), shape[128, 11008]
        // 一个group_id处理256个数据, 一个local_id处理(k0 * 4)个数据(reudce) --> blockIdx = 43, threadIdx = 64, 即每次读取 [1, 4] 个Scale数据 -> [1, 4*32]个Weight的scale
        lv1612_local[0] = vload4(0, lv1612 + (((k_0 * 11008) + ((convert_int(get_group_id(0))) * 256)) + ((convert_int(get_local_id(0))) * 4)));
        for (int k_1 = 0; k_1 < 4; ++k_1) {
        // load w_int4, shape[512, 11008]
        // 一个group_id处理256个数据, 一个local_id处理(k1 * k0 * 4)个数据 --> blockIdx = 43, threadIdx = 64, 即每次读取 [1, 4] 个 Weight数据 -> [1, 4*8]个Wieght的量化数据
        lv1611_local[0] = vload4(0, lv1611 + ((((k_0 * 44032) + (k_1 * 11008)) + ((convert_int(get_group_id(0))) * 256)) + ((convert_int(get_local_id(0))) * 4)));
        // weight: u32 X 4 -> 8 X int4 X 4
        for (int k_2 = 0; k_2 < 8; ++k_2) {
            // half4 ouput = output + (half4 lv1622_shared[k_0*32 + k_1*8 + k_2]) * half4
            var_matmul_intermediate_local[0] = (var_matmul_intermediate_local[0] + (((half4)(lv1622_shared[(((k_0 * 32) + (k_1 * 8)) + k_2)], lv1622_shared[(((k_0 * 32) + (k_1 * 8)) + k_2)], lv1622_shared[(((k_0 * 32) + (k_1 * 8)) + k_2)], lv1622_shared[(((k_0 * 32) + (k_1 * 8)) + k_2)])) * (((convert_half4(((lv1611_local[0]  >>  ((uint4)(((convert_uint(k_2)) * (uint)4), ((convert_uint(k_2)) * (uint)4), ((convert_uint(k_2)) * (uint)4), ((convert_uint(k_2)) * (uint)4))))  &  ((uint4)((uint)15, (uint)15, (uint)15, (uint)15))))) - ((half4)((half)7.000000e+00f, (half)7.000000e+00f, (half)7.000000e+00f, (half)7.000000e+00f))) * lv1612_local[0])));
        }
        }
    }
    vstore4((var_matmul_intermediate_local[0] * (((half4)((half)1.000000e+00f, (half)1.000000e+00f, (half)1.000000e+00f, (half)1.000000e+00f)) / (((half4)((half)1.000000e+00f, (half)1.000000e+00f, (half)1.000000e+00f, (half)1.000000e+00f)) + exp((((half4)((half)0.000000e+00f, (half)0.000000e+00f, (half)0.000000e+00f, (half)0.000000e+00f)) - var_matmul_intermediate_local[0]))))), 0, p_output0_intermediate + (((convert_int(get_group_id(0))) * 256) + ((convert_int(get_local_id(0))) * 4)));
    }
    """ 
    sch = tvm.tir.Schedule(func)
    b0 = sch.get_block(name="decode", func_name="main") # decode([512,11008], [128, 11008]) -> [4096, 11008]
    b1 = sch.get_block(name="matmul", func_name="main") # matmul([4096], [4096, 11008], ) -> [11008]
    # matmul: 1, 1, 11008, 4096
    l2, l3, l4, l5 = sch.get_loops(block=b1)
    # l6 = 11008
    l6 = sch.fuse(l2, l3, l4, preserve_unit_iters=True)
    # v7 = 43, v8 = 64, v9 = 4
    # 搜索记录:
    #   None, 512, 2   2.039464ms
    #   43, 256, 2     1.007784ms
    #   None, 128, 2   1.154256ms
    #   None, 64, 2    1.558120ms

    #   None, 256, 4   1.893800ms
    #   None, 200, 4   0.906336ms
    #   None, 180, 4   2.019104ms
    #   None, 160, 4   1.849120ms
    #   None, 140, 4   2.168464ms
    #   None, 129, 4   2.527432ms
    #   43, 128, 4     0.917504ms  best
    #   None, 100, 4   2.207560ms

    #   86, 64, 4      1.106272ms
    #   172, 32, 4     1.738040ms
    #   None, 128, 8   37.157400ms
    #   None, 80, 8    38.082024ms
    #   43, 64, 8      1.292280ms
    #   None, 50, 8    35.535704ms
    #   86, 32, 8      2.030792ms
    ### 注: None会不关心是否会整除, 会引入判断语句来防止越界
    blockIdxX = None
    threadidxX = 64
    vectorize_factor = 8
    # v7, v8, v9 = sch.sample_perfect_tile(
    #     loop=l6, n=3, max_innermost_factor=4, decision=[blockIdxX, threadidxX, vectorize_factor]
    # )
    # print(sch.get(v7)) #可以打印值
    # spatial axes l6 = 11008 -> l10 = 43, l11 = 64, l12 = 4
    # l10, l11, l12 = sch.split(loop=l6, factors=[v7, v8, v9], preserve_unit_iters=True)
    l10, l11, l12 = sch.split(loop=l6, factors=[blockIdxX, threadidxX, vectorize_factor], preserve_unit_iters=True)

    # l5 = 4096 -> v13 = 128, v14 = 4, v15 = 8
    v13, v14, v15 = sch.sample_perfect_tile(
        loop=l5, n=3, max_innermost_factor=8, decision=[128, 4, 8]
    )
    # reduce l5 = 4096 -> l16 = 128, l17 = 4, l18 = 8
    l16, l17, l18 = sch.split(
        loop=l5, factors=[v13, v14, v15], preserve_unit_iters=True
    )
    # spatial loopRV: l10, l11, l12
    # reduce loopRV: l16, l17, l18
    ## 从这里来看规律：
    ###     spatial可以拆分，内层factor l12可以放到最内层循环
    ###     reduce可以做拆分，并移动到spatial外层循环和内层循环之间，效果上就是每个reduce position可以执行多次spatial操作(即spatial内层)
    ####        优化效果: 1. 在spatial内层操作时，输出的element不变，减少了输出内存的读取；但是对于W来说，需要隔行去读取element, 因此需要多计算几个reduce(列)以利用cache
    ####        优化效果: 2. 由于W shape = [4096, 11008], x[4096] * shape的操作，相当于x与每列W做运算，这会导致cache命中极低，导致cache thrashing(缓存抖动)
    ####            调整后, 对于W的读取，每一个k_inner(l17 * l18 = 4 * 8, fp16数据类型，正好是64bytes)块中会做s_inner(l12 = 4)次运算, 
    ####            ** 相当于4个cache line同时计算, 实际上4行这个数字取决于cache的大小，这里应该可以更大
    ####            ** 相当于，做了tiling操作, 为什么要将k_inner拆分，目前理解是可以用l18(8)来做vectorize的load/store/alu 指令(即SIMD)
    ####        优化效果: 3. 对于外层spatial可以绑定到硬件线程了
    sch.reorder(l10, l11, l16, l17, l18, l12)
    # l10 = 43, 绑定到blockIdx
    sch.bind(loop=l10, thread_axis="blockIdx.x")
    # l11 = 64, 绑定到threadIdx,  64对于adereno gpu来说有意义，因为cache line = 64bytes
    # l10*l11 = 2752, AD4X后的GPU最大的workgroup size(threadIdx.x * threadIdx.y * threadIdx.z) 一般为1024， 
    sch.bind(loop=l11, thread_axis="threadIdx.x")
    # 将decode过程内联到使用decode结果的block中去，这里就是将decode放到matmul中
    ## 从最终优化上来看：将 w = dequant(scale); o = x * w 合并为 => o = x * dequant(scale), 即将语句合并了
    ##      由于gemv中每个w值参与一次计算, 所以这一步可以节省中间缓存的内存搬运消耗，并可以节省memory, 理论上直接计算的中间结果会存在register，访存更快
    ##      @TODO 但是这个计算合并的操作是不是`compute_inline`做的，还不清楚 -- 看`compute_inline`的example是这样的
    ### 在compute_inline之前, block(b1).reads = [lv1622, var_decode_intermediate], writes = [var_matmul_intermediate]
    sch.compute_inline(block=b0)
    ## block(b1).reads = [lv1622, lv1611, lv1612], writes = [var_matmul_intermediate]
    ## 从reads的顺序来看, compute_inline会把b0.reads插到b1.reads后面,  同理writes应该也是这样, 但是这里没有writes, 所以没体现
    ### 也就是compute_inline把var_decode_intermediate中间变量给省去了，印证了之前的猜测
    ## b19指向的是 var_matmul_intermediate的赋值过程，即从global memory -> local memroy
    ## 也就是b19是一个block, 并且包含 loops, b19.loops(未经调整)=b19.shape
    b19 = sch.cache_write(block=b1, write_buffer_index=0, storage_scope="local")
    # reverse_compute_at 将一个消费者block(b19)移动到指定的loop(l11)中，并且重新生成该block包含的loops(暂时理解为绑定到该block的loops)
    ##   这样该block(b19)消费的buffer(write index 0, 即var_matmul_intermediate)区域就可以覆盖
    ##    指定loop(l11)下生产者blocks(一个或多个，这里指decode)生成的buffer区域(暂时理解为var_decode_intermediate)
    # b19: [1, 1, 11008]
    # b19插入后: [l10, l11, 1, 1, 1*1*11008/(l10*l11)] -> [43, 64, 1, 1, 4] 这个只是纯赋值的loop grid，在update时还要将reduce插入
    sch.reverse_compute_at(block=b19, loop=l11, preserve_unit_loops=True, index=-1)
    # 执行完reverse_compute_at, b1.reads和writes的内容没变
    # 将lv1611[512,11008]设置为local内存
    ## 这个内存设置奇怪, 只用一次, 不需要线放到local中
    b20 = sch.cache_read(block=b1, read_buffer_index=1, storage_scope="local")
    # 将lv1612[128,11008]设置为local内存
    ## 这个内存设置奇怪, 只用一次, 不需要线放到local中
    b21 = sch.cache_read(block=b1, read_buffer_index=2, storage_scope="local")
    # 将lv1622[1,1,4096]设置为shared内存
    b22 = sch.cache_read(block=b1, read_buffer_index=0, storage_scope="shared")
    # 将一个生产者block(b22)移动到指定loop(l11)中，并且重新生成该block(b22)包含的loops, 这样该block(b22)生产的buffer区域就可以覆盖指定loop(l11)下消费者blocks消费的buffer区域
    ## preserve_unit_loops的作用: 如将(128, 129) compute_at (128)下，若为true则会有 for 128 (for(1, 129)), 即保留shape中value为1的loop, 若为False, 则有for 128 (for(129))
    # b22: lv1622[1,1,4096]
    ## 猜测shape: [l10, l11, 1, 1, 4096/(l10 * l11)] @TODO 无法整除咋整.. 无法整除就没整除 -> [l10, l11, 1, 1, 4096], 后面重新划分4096, 重新绑定threadIdx.x, 再者shared_memory不能绑定到blockIdx中, 所以必须重新绑定
    ## 观察到shape: [[l10, l11], [1,1,4096]] 即b22的初始化还在[l10,l11]的loop内，但是压根没按照这个计算, 这种情况得看下最终的cuda代码是如何处理的
    ##### for i0_i1_i2_fused_1 in T.thread_binding(T.int64(64), thread="threadIdx.x"):
    #####     lv1622_shared = T.Buffer((T.int64(1), T.int64(1), T.int64(4096)), "float16", scope="shared")
    #####     for ax0, ax1, ax2 in T.grid(T.int64(1), T.int64(1), T.int64(4096)):
    #####         with T.block("lv1622_shared"):
    #####             v0, v1, v2 = T.axis.remap("SSS", [ax0, ax1, ax2])
    #####             lv1622 = T.Buffer((T.int64(1), T.int64(1), T.int64(4096)), "float16")
    #####             T.reads(lv1622[v0, v1, v2])
    #####             T.writes(lv1622_shared[v0, v1, v2])
    #####             lv1622_shared[v0, v1, v2] = lv1622[v0, v1, v2]
    sch.compute_at(block=b22, loop=l11, preserve_unit_loops=True, index=-1)
    # print(sch.get(l11))
    v23 = sch.sample_categorical(
        candidates=[1, 2, 4, 8], probs=[0.25, 0.25, 0.25, 0.25], decision=3
    )
    # 做标注，实际形式会在b22中生成 T.block_attr({"ann_key", "ann_value"})
    ## 标注了 lv1622[4096] cooperative_fetch:8
    ## 在opencl kernel中实现了每个block中,不同ThreadIdx, 通过vload8加载数据到shared中
    sch.annotate(
        block_or_loop=b22, ann_key="meta_schedule.cooperative_fetch", ann_val=v23
    )
    # 这里有点奇怪，b20和b21是decode需要用到的buffer, 而l16 l17是reduce loop拆分的前两个subloop, 为什么会绑定到不同
    sch.compute_at(block=b20, loop=l17, preserve_unit_loops=True, index=-1)
    sch.compute_at(block=b21, loop=l16, preserve_unit_loops=True, index=-1)
    # @TODO 猜测b20_0: b20的原始loop: [512, 11008], 插入到l17后变为: [l10, l11, l16, l17, 512, 11008]
    # l10 = 43, l11 = 64, l16 = 128, l17 = 4 => [l10, l11, l16, l17, 1, 512*11008/(43*64*128*4)]
    ## 实际情况: 插入l17后变为 [43, 64, 128, 4, 1, 4]
    l24, l25, l26, l27, l28, l29 = sch.get_loops(block=b20)
    # print(sch.get(l28))#, sch.get(l25),sch.get(l26), sch.get(l27), sch.get(l28),sch.get(l29))
    # vectorize 4
    sch.vectorize(loop=l29)
    # b21: lv1612[128,11008]
    # b21 插入后: [l10, l11, l16, 1, 128*11008/(l10*l11*l16)] => [43, 64, 128, 1, 4 ]
    l30, l31, l32, l33, l34 = sch.get_loops(block=b21)
    sch.vectorize(loop=l34)
    # b19: [1, 1, 11008]
    # b19插入后: [l10, l11, 1, 1, 11008/(l10*l11)] -> [43, 64, 1, 1, 4]
    l35, l36, l37, l38, l39 = sch.get_loops(block=b19)
    # l39 = 4
    sch.vectorize(loop=l39)
    # l12 = 4
    sch.vectorize(loop=l12)
    # 包含reduction计算的即为reduction block
    # 将reduction block中的 Init block和update block拆分， 即将matmul中var_matmul_intermediate = float16(0)的操作拆分出来
    # 注意现在b1通过之前的compute_at包含了很多初始化的计算(@TODO 实际看只有b22的赋初始值的过程)
    # b40是b22的赋初始值过程 @TODO 没找到b20 b21 b22的内存转移操作在哪儿
    b40 = sch.decompose_reduction(block=b1, loop=l16)
    b41 = sch.get_block(name="T_update", func_name="main")
    sch.reverse_compute_inline(block=b41)
    # 标注scheudle的后处理开始，@TODO 不知道作用 
    sch.enter_postproc()
    # 去掉 annotate, 从这里可以猜测`enter_postproc`应该对annotate信息做了处理
    sch.unannotate(block_or_loop=b22, ann_key="meta_schedule.cooperative_fetch")
    # 此时b22只包含 global memory -> shared memory的过程
    # 察到shape: [[l10, l11], [1,1,4096]] 即b22的初始化还在[l10,l11]的loop内
    l43, l44, l45, l46, l47 = sch.get_loops(block=b22)
    # preserve_unit_iters作用不大，仅是否考虑计算 value为1 的loop
    # 用于vectorize构建shared_memory
    # l48, l49, l50 = sch.split(loop=l47, factors=[None, 64, 8], preserve_unit_iters=True)
    l48, l49, l50 = sch.split(loop=l47, factors=[None, threadidxX, 8], preserve_unit_iters=True)
    # l49 = 64, l50 = 8
    sch.vectorize(loop=l50)
    sch.bind(loop=l49, thread_axis="threadIdx.x")
    return sch.mod["main"].with_attr("tir.is_scheduled", 1)


sch_manual = tvm.tir.Schedule(ModuleToManual)
# sch_fused_decode5_fused_matmul6_silu1(sch_manual.mod[func_name])
sch_manual.mod['main'] = sch_fused_decode5_fused_matmul6_silu1(sch_manual.mod[func_name])
# print(sch_manual.mod.script())
print("================================================")
rt_mod = tvm.build(sch_manual.mod, target="opencl")
print(rt_mod.imported_modules[0].get_source())

// Function: main_kernel
#ifdef cl_khr_fp16
#pragma OPENCL EXTENSION cl_khr_fp16 : enable
#elif defined(cl_amd_fp16)
#pragma OPENCL EXTENSION cl_amd_fp16 : enable
#else
#error "Half precision floating point not supported by OpenCL implementation on your device." 
#endif

__kernel void main_kernel(__global uint* restrict lv1611, __global half* restrict lv1612, __global half* restrict lv1622, __global half* restrict p_output0_intermediate) {
  __local half lv1622_shared[4096];
  half8 var_matmul_intermediate_local[1];
  half8 lv1612_local[1];
  uint8 lv1611_local[1];
  for (int ax2_0 = 0; ax2_0 < 8; ++ax2_0) {
    vstore8(vload8(0, lv1622 + ((ax2_0 * 512) + ((convert_int(get_local_id(0))) * 8))), 0, lv1622_shared + ((ax2_0 * 512) + ((convert_int(get_local_id(0))) * 8)));
  }
  var_matmul_intermediate_local[0] = ((half8)((half)0.000000e+00f, (half)0.000000e+00f, (half)0.000000e+00f, (half)0.000000e+00f, (half)0.000000e+00f, (half)0.000000e+00f, (half)0.000000e+00f, (half)0.000000e+00f));


In [4]:
# run and compare with cuda
import numpy as np
def _detect_local_cuda():
    dev = tvm.cuda()
    if not dev.exist:
        return None
    return tvm.target.Target(
        {
            "kind": "cuda",
            "max_shared_memory_per_block": dev.max_shared_memory_per_block,
            "max_threads_per_block": dev.max_threads_per_block,
            "thread_warp_size": dev.warp_size,
            "registers_per_block": 65536,
            "arch": "sm_" + tvm.cuda().compute_version.replace(".", ""),
        }
    )
# target = tvm.target.Target("cuda", host="llvm")
target = _detect_local_cuda()

print(target)
# 定义计算任务
dev = tvm.cuda(0)

num_flop = 1228406784
W_w_np = np.random.uniform(size=(w_w_x, w_y)).astype("uint32")
W_s_np = np.random.uniform(size=(w_s_x, w_y)).astype("float16")
Input_np = np.random.uniform(size=(1, 1, x_shape)).astype("float16")
# W_w_np = np.ones((w_w_x, w_y), np.uint32) * 1#.astype("uint32")
# W_s_np = np.ones((w_s_x, w_y), np.float16) * 1#.astype("float16") * 2
# Input_np = np.ones((1, 1, x_shape), np.float16)#.astype("float16")
Output_nd = tvm.nd.array(np.zeros((1, 1, w_y), dtype="float16"), dev)
def numpy_caculate():
    test_cols = 10
    output = np.zeros((1, 1, test_cols), dtype = np.float16)
    W_w_inv_np = np.transpose(W_w_np)
    W_s_inv_np = np.transpose(W_s_np)
    for i in range(test_cols):
        for r in range(x_shape):
            temp = Input_np[0][0][r] * np.float16((W_w_inv_np[i][r // 8] >> ((r % 8) * 4) & (15)) - np.float16(7.0)) * W_s_inv_np[i][r // 32]
            output[0][0][i] = output[0][0][i] + temp
    print(output)
    output = np.zeros((1, 1, test_cols), dtype = np.float16)
    for i in range(test_cols):
        for r in range(x_shape):
            temp = Input_np[0][0][r] * np.float16((W_w_np[r // 8][i] >> ((r % 8) * 4) & (15)) - np.float16(7.0)) * W_s_np[r // 32][i]
            temp_output = output[0][0][i]
            output[0][0][i] = temp_output + temp
            # print(f"{temp_output} + {temp} = {output[0][0][i]}")
    print(output)
numpy_caculate()
def print_npdata(np_data: np.ndarray) :
    d = np_data.flatten()
    p_size = 10 if d.size > 10 else d.size
    print(d[:p_size])

cuda -keys=cuda,gpu -arch=sm_61 -max_num_threads=1024 -max_shared_memory_per_block=49152 -max_threads_per_block=1024 -registers_per_block=65536 -thread_warp_size=32
[[[-6644. -7192. -6284. -6688. -6772. -6120. -6876. -6168. -6472. -6424.]]]
[[[-6644. -7192. -6284. -6688. -6772. -6120. -6876. -6168. -6472. -6424.]]]


In [25]:
# cuda未优化版本测试
sch = tvm.tir.Schedule(ModuleSrc)
with target:
    src_gpu_mod = tvm.tir.transform.DefaultGPUSchedule()(sch.mod) ##
rt_mod = tvm.build(src_gpu_mod, target="cuda")
W_w_nd = tvm.nd.array(W_w_np, dev)
W_s_nd = tvm.nd.array(W_s_np, dev)
Input_nd = tvm.nd.array(Input_np, dev)
Output_nd = tvm.nd.array(np.zeros((1, 1, w_y), dtype="float16"), dev)
evaluator = rt_mod.time_evaluator("main", dev, number=100)
print("manual_evaluator GEMV-Blocking: %f GFLOPS" % (num_flop / evaluator(W_w_nd, W_s_nd, Input_nd, Output_nd).mean / 1e9))
# print(Output_nd.numpy())
print_npdata(Output_nd.numpy())

manual_evaluator GEMV-Blocking: 65.352494 GFLOPS
[-5976. -6344. -7012. -5560. -6352. -5812. -7388. -6224. -6852. -6048.]


In [8]:
os.environ["TVM_NDK_CC"]="/home/sensetime/Android/Sdk/ndk/25.2.9519653/toolchains/llvm/prebuilt/linux-x86_64/bin/aarch64-linux-android33-clang++"
target = tvm.target.Target("opencl -device=adreno", host="llvm -mtriple=aarch64-linux-gnu")
device_key="android"
rpc_host = "10.4.236.32"
rpc_port = 9190
comp_target = tvm.target.Target("opencl", host="llvm -mtriple=aarch64-linux-android")  # TODO: Only support arm64 for now

def test_opencl(mod: tvm.IRModule, name_hint: str):
    # mod = tvm.lower(sch_manual.mod)
    print("Build ...")
    android_rt_mod = tvm.build(mod, target="opencl", target_host="llvm -mtriple=aarch64-linux-android")
    # print(android_rt_mod.imported_modules[0].get_source())
    temp = utils.tempdir()
    path_dso_cl = temp.relpath("dev_lib_cl.so")
    android_rt_mod.export_library(path_dso_cl, ndk.create_shared)

    print("Run GPU(OpenCL Flavor) test ...")
    # Establish remote connection with target hardware

    tracker = rpc.connect_tracker(rpc_host, rpc_port)
    remote = tracker.request(device_key, priority=0, session_timeout=60)
    print("Connect to device done.")
    dev = remote.cl(0)
    remote.upload(path_dso_cl)
    f1 = remote.load_module("dev_lib_cl.so")

    W_w_nd = tvm.nd.array(W_w_np, dev)
    W_s_nd = tvm.nd.array(W_s_np, dev)
    Input_nd = tvm.nd.array(Input_np, dev)
    Output_nd = tvm.nd.array(np.zeros((1, 1, w_y), dtype="float16"), dev)
    test_number=32
    time_f = f1.time_evaluator(f1.entry_name, dev, number=test_number)
    cost = time_f(W_w_nd, W_s_nd, Input_nd, Output_nd).mean
    print("evaluator[%s] GEMV-Blocking: %fms with loop %d" % (name_hint, cost * 1000, test_number))
    print("evaluator[%s] GEMV-Blocking: %fGFLOPS" % (name_hint, num_flop / cost / 1e9))

    return Output_nd.numpy()

In [27]:
# Licensed to the Apache Software Foundation (ASF) under one
# or more contributor license agreements.  See the NOTICE file
# distributed with this work for additional information
# regarding copyright ownership.  The ASF licenses this file
# to you under the Apache License, Version 2.0 (the
# "License"); you may not use this file except in compliance
# with the License.  You may obtain a copy of the License at
#
#   http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing,
# software distributed under the License is distributed on an
# "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY
# KIND, either express or implied.  See the License for the
# specific language governing permissions and limitations
# under the License.
"""A rule for GEMV and DecodeGEMV."""
import re
from functools import reduce
from typing import List, Optional, Union

from tvm import DataType, arith, ir, tir
from tvm.target import Target

from tvm.dlight.base import (
    BlockInfo,
    ScheduleRule,
    collect_vars_used_in_access_region,
    detect_dominant_read,
    is_broadcast_epilogue,
    normalize_prim_func,
    try_inline_contiguous_spatial,
)


def _get_reduction_expr(block: tir.Block) -> Optional[tir.PrimExpr]:
    # Detect and return `Y` in `X[...] = X[...] + Y`
    buffer_store = block.body
    if not isinstance(buffer_store, tir.BufferStore):
        return None
    if not isinstance(buffer_store.value, tir.Add):
        return None
    if not ir.structural_equal(
        buffer_store.value.a,
        tir.BufferLoad(buffer_store.buffer, block.body.indices),
        map_free_vars=True,
    ):
        return None
    return buffer_store.value.b


def get_bytes(dtype: Union[DataType, str]) -> int:
    num = re.findall(r"\d+", dtype)
    if len(num) != 1:
        raise ValueError(f"Cannot get bytes from {dtype}")
    return int(num[0]) // 8


def is_gemv(sch: tir.Schedule, block_info: BlockInfo) -> Optional[List[tir.Buffer]]:
    """Check if the block is a GEMV.

    Parameters
    ----------

    sch : tir.Schedule
        The schedule

    block_info : BlockInfo
        The block info to be checked


    Returns
    -------
    ret : Optional[List[tir.Buffer]]
        The vector buffers used in the GEMV if it is a GEMV, otherwise None.
    """
    block = block_info.block_rv
    block_stmt = sch.get(block)
    conditions = []
    conditions.append(block_info.is_reduction())
    conditions.append(len(block_stmt.reads) >= 2)
    conditions.append(len(block_stmt.writes) == 1)
    conditions.append(_get_reduction_expr(block_stmt) is not None)
    conditions.append(len(collect_vars_used_in_access_region(block_stmt.writes[0].region)) > 0)
    if not all(conditions):
        return None

    iter_num = len(block_stmt.iter_vars)
    ret = [
        read.buffer
        for read in block_stmt.reads
        if len(collect_vars_used_in_access_region(read.region)) < iter_num
    ]
    return ret if 0 < len(ret) < len(block_stmt.reads) else None


def normalize(
    sch: tir.Schedule,
    block_info: BlockInfo,
) -> Optional[bool]:
    """Normalize the main block."""
    block_stmt: tir.Block = sch.get(block_info.block_rv)
    access = arith.normalize_to_iter_sum(
        detect_dominant_read(block_stmt),
        input_iters={i.var: i.dom for i in block_stmt.iter_vars},
    )

    buffers_use_vars = [collect_vars_used_in_access_region(buf.region) for buf in block_stmt.writes]
    buffers_use_vars.extend(
        [collect_vars_used_in_access_region(buf.region) for buf in block_stmt.reads]
    )
    if access.base != 0:
        print("access.base == 1")
        return None
    iter_to_info = {i.var: i for i in block_info.iters}
    batch_loops, s_loops, r_loops, c_loops = [], [], [], []
    inner_axis = access.args[-1].source.source
    print(f"inner_axis = {inner_axis}")
    is_inner_reduction = iter_to_info[inner_axis].kind == "R"

    for split_expr in access.args:
        var = split_expr.source.source
        info = iter_to_info.get(var)
        loop = info.loop_rv
        is_reduction = info.kind == "R"
        if split_expr.lower_factor > 1:
            if c_loops:
                print(f"c_loops = {c_loops}")
                return None
            loop, c_loop = sch.split(loop, factors=[None, split_expr.lower_factor])
            # we only support the inner most dim being grouped atm
            if is_reduction ^ is_inner_reduction:
                print(f"is_reduction = {is_reduction}, is_inner_reduction = {is_inner_reduction}")
                return None
            c_loops.append(c_loop)
        if is_reduction:
            r_loops.append(loop)
        elif all([var in buf_vars for buf_vars in buffers_use_vars]):
            batch_loops.append(loop)
        else:
            s_loops.append(loop)

    assert s_loops
    assert r_loops
    if not c_loops:
        c_loops = [sch.add_unit_loop(block_info.block_rv)]
    if not batch_loops:
        batch_loops = [sch.add_unit_loop(block_info.block_rv)]
    sch.reorder(*batch_loops, *s_loops, *r_loops, *c_loops)
    sch.fuse(*batch_loops)
    sch.fuse(*s_loops)
    sch.fuse(*r_loops)
    return is_inner_reduction


class GEMV(ScheduleRule):
    """A rule for GEMV and DecodeGEMV."""

    def apply(  # pylint: disable=too-many-locals,too-many-branches,too-many-return-statements
        self,
        func: tir.PrimFunc,
        target: Target,
        _: bool,
    ) -> Union[None, tir.Schedule, List[tir.Schedule]]:
        print("GEMV in!")
        if not isinstance(func, tir.PrimFunc):
            return None
        sch = tir.Schedule(func)
        block_infos = normalize_prim_func(sch)
        block_infos = try_inline_contiguous_spatial(sch, block_infos)
        if len(block_infos) == 1:
            epilogue = None
        elif len(block_infos) == 2:
            epilogue = block_infos[1]
            if not epilogue.is_injective():
                return None
        else:
            return None

        block_info = block_infos[0]
        if len(block_info.iters) not in [2, 3]:
            # either [B, S, R] = [B, S, R] * [B, R]
            # or [S, R] = [S, R] * [R]
            return None
        block = block_info.block_rv
        vector_input_buffers = is_gemv(sch, block_info)
        if vector_input_buffers is None:
            return None
        print(f"block_infos len: {len(block_infos)}")
        for b in block_infos:
            print(f"\t{b}")
        print("================================================")
        # Step 1. Normalize the block, merge spatial and reduction iters
        is_inner_reduction = normalize(sch, block_info)
        print(is_inner_reduction)

        # Step 2. Do the scheduling
        if is_inner_reduction:
            self.sch_inner_reduction(sch, target, block, vector_input_buffers, epilogue)
            return sch
        else:
            # TODO: Need to handle GEMV with KN layout
            return None

    def sch_inner_reduction(  # pylint: disable=too-many-arguments, invalid-name, unused-argument
        self,
        sch: tir.Schedule,
        target: Target,
        block: tir.schedule.BlockRV,
        vector_input_buffers: List[tir.Buffer],
        epilogue_info: Optional[BlockInfo],
    ):
        """Schedule the inner reduction block."""

        def get_max_factor(n, factors):
            factors = sorted(factors, reverse=True)
            for factor in factors:
                if n % factor == 0:
                    return factor
            return 1

        def apply(
            sch: tir.Schedule,
            gemv,
            TAG_S,
            TAG_R,
            TS,
            TR,
            TILE_S,
            TILE_R,
            VEC_LOAD,
            VEC_C,
            LOAD_V_SHARED,
            LOAD_V_VEC,
            UNROLL,
        ):
            # rfactor: reduce to tx * vec_c
            _, s, r, c = sch.get_loops(block=gemv)
            s = sch.fuse(_, s)
            r = sch.fuse(r, c)
            bx, ts, tile_s = sch.split(s, factors=[None, TS, TILE_S], preserve_unit_iters=True)
            r, tr, tile_r_vec_n, vec_c = sch.split(
                r, factors=[None, TR, TILE_R // VEC_C, VEC_C], preserve_unit_iters=True
            )
            sch.reorder(r, tile_r_vec_n, tr, vec_c)
            tr_vec_c = sch.fuse(tr, vec_c)
            rf = sch.rfactor(tr_vec_c, 0)

            # rfactor: reduce to tx
            bx, ts, tile_s, tr_vec_c = sch.get_loops(block=gemv)
            tr, vec_c = sch.split(tr_vec_c, factors=[TR, None], preserve_unit_iters=True)
            rf2 = sch.rfactor(tr, 0)

            # bind, vectorize compute
            bx, ts, tile_s, r, tile_r_vec_n, tr_vec_c = sch.get_loops(block=rf)
            tr, vec_c = sch.split(tr_vec_c, factors=[TR, None], preserve_unit_iters=True)
            sch.reorder(bx, ts, tr, r, tile_s, tile_r_vec_n, vec_c)
            sch.bind(bx, "blockIdx.x")
            sch.bind(ts, TAG_S)
            sch.bind(tr, TAG_R)
            sch.vectorize(vec_c)

            shared_mem_usage = 0
            for buf in vector_input_buffers:
                buf_size = reduce(
                    lambda x, y: x * y, buf.shape, tir.IntImm(buf.shape[0].dtype, 1)
                ) * get_bytes(buf.dtype)
                shared_mem_usage += buf_size
            LOAD_V_SHARED = (
                LOAD_V_SHARED
                and isinstance(shared_mem_usage, tir.IntImm)
                and shared_mem_usage.value <= target.max_shared_memory_per_block
            )

            # vectorize load A
            # (TODO) this is now actually problematic since the number of loops is dependent on the
            # number of dimensions of A_q
            Aq_local = sch.cache_read(rf, read_buffer_index=1, storage_scope="local")
            sch.compute_at(Aq_local, r, preserve_unit_loops=True)
            s_local, r_local = sch.get_loops(block=Aq_local)[-2:]
            s_local, vec_load = sch.split(
                s_local, factors=[None, VEC_LOAD], preserve_unit_iters=True
            )
            sch.reorder(s_local, r_local, vec_load)  # either s_local or r_local should be 1
            sch.vectorize(vec_load)

            # load vector into shared memory, shape should be the whole vector
            if LOAD_V_SHARED:
                assert len(vector_input_buffers) == 1
                V_shared = sch.cache_read(rf, read_buffer_index=0, storage_scope="shared")
                sch.compute_at(V_shared, tr, preserve_unit_loops=True)
                l = sch.get_loops(block=V_shared)[-1]
                loop: tir.For = sch.get(l)
                if isinstance(loop.extent, tir.IntImm):
                    # avoid introducing predicates when vector length is too large
                    vec_length = max(
                        min(
                            get_max_factor(
                                (int)(loop.extent),
                                [TS * TR * 1, TS * TR * 2, TS * TR * 4, TS * TR * 8],
                            )
                            // TS
                            // TR,
                            LOAD_V_VEC,
                        ),
                        1,
                    )
                else:
                    vec_length = LOAD_V_VEC
                if TAG_R == "threadIdx.x":
                    _, ty, tx, vec = sch.split(
                        l, factors=[None, TS, TR, vec_length], preserve_unit_iters=True
                    )
                else:
                    _, ty, tx, vec = sch.split(
                        l, factors=[None, TR, TS, vec_length], preserve_unit_iters=True
                    )
                sch.bind(ty, "threadIdx.y")
                sch.bind(tx, "threadIdx.x")
                sch.vectorize(vec)

            # reduce tile_s * tr * vec to tile_s * tr
            sch.reverse_compute_at(rf2, loop=bx, preserve_unit_loops=True)
            tr, vec_c, *ts_tile_s = sch.get_loops(block=rf2)[1:]
            ts_tile_s = sch.fuse(*ts_tile_s)
            ts, tile_s = sch.split(ts_tile_s, factors=[TS, None], preserve_unit_iters=True)
            tile_s, vec_s = sch.split(
                tile_s,
                factors=[None, get_max_factor(TILE_S, [1, 2, 4, 8])],
                preserve_unit_iters=True,
            )
            sch.reorder(ts, tr, tile_s, vec_s, vec_c)
            sch.bind(ts, TAG_S)
            sch.bind(tr, TAG_R)
            sch.vectorize(vec_s)

            # reduce tile_s * tr to tile_s
            sch.reverse_compute_at(gemv, loop=bx, preserve_unit_loops=True)
            tr, *ts_tile_s = sch.get_loops(block=gemv)[1:]
            ts_tile_s = sch.fuse(*ts_tile_s)
            ts, tile_s = sch.split(ts_tile_s, factors=[TS, None], preserve_unit_iters=True)
            sch.reorder(tile_s, ts, tr)
            sch.bind(ts, TAG_S)
            sch.bind(tr, TAG_R)

            sch.decompose_reduction(rf, loop=sch.get_loops(block=rf)[3])
            sch.decompose_reduction(rf2, loop=sch.get_loops(block=rf2)[-1])

            sch.set_scope(rf, buffer_index=0, storage_scope="local")
            sch.set_scope(rf2, buffer_index=0, storage_scope="local")

            unroll_factor = UNROLL

            sch.annotate(
                block_or_loop=sch.get_loops(rf)[3],
                ann_key="pragma_auto_unroll_max_step",
                ann_val=unroll_factor,
            )
            sch.annotate(
                block_or_loop=sch.get_loops(rf)[3], ann_key="pragma_unroll_explicit", ann_val=1
            )

            sch.annotate(
                block_or_loop=sch.get_loops(rf2)[3],
                ann_key="pragma_auto_unroll_max_step",
                ann_val=unroll_factor,
            )
            sch.annotate(
                block_or_loop=sch.get_loops(rf2)[3], ann_key="pragma_unroll_explicit", ann_val=1
            )

            if LOAD_V_SHARED:
                sch.annotate(
                    block_or_loop=sch.get_loops(V_shared)[-4],
                    ann_key="pragma_unroll_explicit",
                    ann_val=unroll_factor,
                )
                sch.annotate(
                    block_or_loop=sch.get_loops(V_shared)[-4], ann_key="pragma_vectorize", ann_val=1
                )

            # Schedule epilogue
            if epilogue_info is not None:
                epilogue = epilogue_info.block_rv
                if is_broadcast_epilogue(sch, block, epilogue):
                    sch.reverse_compute_at(epilogue, bx)
                    sch.set_scope(block, 0, "shared")
                    _, _, *s = sch.get_loops(epilogue)  # pylint: disable=invalid-name
                    _, tx = sch.split(sch.fuse(*s), factors=[None, TX])
                    sch.bind(tx, "threadIdx.x")
                else:
                    sch.reverse_compute_at(epilogue, bx, preserve_unit_loops=True)
                    ts_tile_s = sch.fuse(*sch.get_loops(epilogue)[1:])
                    ts_tile_s = sch.get_loops(epilogue)[-1]
                    ts, tile_s = sch.split(ts_tile_s, factors=[TS, None], preserve_unit_iters=True)
                    sch.bind(ts, TAG_S)
                    sch.set_scope(block, 0, "local")
            # pylint: enable=invalid-name
            return sch

        def get_extent(loop_rv: tir.schedule.LoopRV):
            loop: tir.For = sch.get(loop_rv)
            return loop.extent.value if isinstance(loop.extent, tir.IntImm) else loop.extent

        # Specify the `len_tx` and `len_ty` according to the loop extent
        batch, s, r, c = sch.get_loops(block=block)
        len_batch, len_s, len_r, len_c = (
            get_extent(batch),
            get_extent(s),
            get_extent(r),
            get_extent(c),
        )
        len_S = len_batch * len_s
        len_R = len_r * len_c

        TAG_S, TAG_R = "threadIdx.y", "threadIdx.x"
        print(f"kind.name = {target.kind.name}, host = {target.host}")
        if target.kind.name == "cuda":
            VEC_C = 4
            LOAD_V_SHARED = True
            LOAD_V_VEC = 8
            UNROLL = 256
            if isinstance(len_S, int):
                if len_S > len_R:
                    TS, TR = 4, 64
                else:
                    TS, TR = 16, 32
        elif target.kind.name == "metal":
            VEC_C = 2
            LOAD_V_SHARED = True
            LOAD_V_VEC = 4
            UNROLL = 256
            TS, TR = 64, 8
        elif target.kind.name == "rocm":
            VEC_C = 4
            LOAD_V_SHARED = True
            LOAD_V_VEC = 8
            UNROLL = 256
            if isinstance(len_S, int):
                if len_S > len_R:
                    TS, TR = 1, 128
                else:
                    TS, TR = 8, 64
        elif target.kind.name == "opencl" and "android" in str(target.host):
            print("dlight GEMV: opencl")
            TAG_S, TAG_R = "threadIdx.x", "threadIdx.y"
            VEC_C = 8
            LOAD_V_SHARED = True
            LOAD_V_VEC = 4
            UNROLL = 8
            TS, TR = 4, 32
        elif target.kind.name == "vulkan":
            VEC_C = 4
            LOAD_V_SHARED = True
            LOAD_V_VEC = 4
            UNROLL = 256
            if isinstance(len_S, int):
                if len_S > len_R:
                    TS, TR = 4, 32
                else:
                    TS, TR = 16, 32
        else:
            print("dlight GEMV: default")
            VEC_C = 1
            LOAD_V_SHARED = False
            LOAD_V_VEC = -1
            UNROLL = 64
            TS, TR = 1, 64

        if not isinstance(len_S, int):
            TS, TR = 1, 64
        TILE_S, TILE_R = (
            1,
            len_c
            if len_c > 1
            else max(get_max_factor(len_r, [TR * 1, TR * 2, TR * 4, TR * 8]) // TR, 1),
        )
        VEC_C = min(get_max_factor(TILE_R, [1, 2, 4, 8]), VEC_C)
        VEC_LOAD = 1

        return apply(
            sch,
            gemv=block,
            TAG_S=TAG_S,
            TAG_R=TAG_R,
            TS=TS,
            TR=TR,
            TILE_S=TILE_S,
            TILE_R=TILE_R,
            VEC_LOAD=VEC_LOAD,
            VEC_C=VEC_C,
            LOAD_V_SHARED=LOAD_V_SHARED,
            LOAD_V_VEC=LOAD_V_VEC,
            UNROLL=UNROLL,
        )


In [28]:
# 未优化版本opencl测试
from tvm import dlight as dl
sch = tvm.tir.Schedule(ModuleSrc)
with target:
    # src_gpu_mod = tvm.tir.transform.DefaultGPUSchedule()(sch.mod) ##
    mod_deploy = dl.ApplyDefaultSchedule(  # pylint: disable=not-callable
        dl.gpu.Matmul(),
        dl.gpu.GEMV(),
        dl.gpu.Reduction(),
        dl.gpu.GeneralReduction(),
        dl.gpu.Fallback(),
    )(sch.mod)
src_output = test_opencl(mod_deploy, "source")
print_npdata(src_output)


Build ...
Run GPU(OpenCL Flavor) test ...


/data/workspace/llm/github/new_wksp/tvm-unity/python/tvm/driver/build_module.py:264: UserWarning: target_host parameter is going to be deprecated. Please pass in tvm.target.Target(target, host=target_host) instead.
  warnings.warn(


Connect to device done.
evaluator[source] GEMV-Blocking: 4.269120 ms with loop 32
evaluator[source] GEMV-Blocking: 287.742388 GFLOPS
[-6600. -6816. -7668. -6056. -6908. -6464. -7976. -6960. -7600. -6672.]


In [113]:
#优化版本opencl测试
# print(sch_manual.mod)
opt_output = test_opencl(sch_manual.mod, "opted")
print_npdata(opt_output)
# np.testing.assert_equal(opt_output, src_output)

Build ...
Run GPU(OpenCL Flavor) test ...
Connect to device done.
evaluator[opted] GEMV-Blocking: 1.295184ms with loop 32
evaluator[opted] GEMV-Blocking: 948.441908GFLOPS
[-6644. -7192. -6284. -6688. -6772. -6116. -6876. -6164. -6472. -6416.]


In [ ]:
import numpy as np
target = tvm.target.Target("opencl -device=adreno", host="llvm -mtriple=aarch64-linux-gnu")
device_key="android"
rpc_host = "10.158.176.30"
rpc_port = 5001
# remote = autotvm.measure.request_remote(device_key, "10.158.176.30", 5001, timeout=10000)
# dev = remote.device(str(target), 0)

# num_flop = 1228406784
# W_np = np.random.uniform(size=(512, vocab_size)).astype("uint32")
# S_np = np.random.uniform(size=(128, vocab_size)).astype("float16")
# Input_np = np.random.uniform(size=(1, 1, 4096)).astype("float16")
# # Output_np = np.random.uniform(size=(1, 1, 4096)).astype("float16")
# W_nd = tvm.nd.array(W_np, dev)
# S_nd = tvm.nd.array(S_np, dev)
# Input_nd = tvm.nd.array(Input_np, dev)
# Output_nd = tvm.nd.array(np.zeros((1, 1, vocab_size), dtype="float32"), dev)

In [ ]:
rpc_config = ms.runner.RPCConfig(tracker_host=rpc_host, tracker_port=rpc_port, tracker_key = device_key)
runner= ms.runner.RPCRunner(rpc_config)
# ms.builder.LocalBuilder()
sch = tvm.tir.Schedule(ModuleSrc)
database = ms.tune_tir(
    mod=ModuleSrc,
    target=target,
    max_trials_global=64,
    num_trials_per_iter=64,
    work_dir="./tune_first",
    cost_model="xgb",
    runner = runner
)
print(len(database))
sch1 = ms.tir_integration.compile_tir(database, sch.mod, target)
print(type(sch1))

In [ ]:
from tvm.script import relax as R
@I.ir_module
class Module:
    @R.function
    def main(A: R.Tensor((3, 4), dtype="float32"), B: R.Tensor((4, 5), dtype="float32")):
        with R.dataflow():
            lv: R.Tensor((3, 5), dtype="float32") = R.matmul(A, B)
            gv: R.Tensor((3, 5), dtype="float32") = lv
            R.output(gv)
        return gv

In [ ]:
## auto_scheduler test
from tvm import auto_scheduler
import numpy as np
a_np = np.random.rand(3, 4).astype("float32")
b_np = np.random.rand(4, 5).astype("float32")
a_nd = tvm.runtime.NDArray(a_np)
b_nd = tvm.runtime.NDArray(b_np)
sch = tvm.tir.Schedule(Module)

params = {"A": a_np, "B": b_np}
## 报错，这里只支持relay
# tasks = auto_scheduler.extract_tasks(sch.mod, params, target=target)
tasks = ms.relax_integration.extract_tasks(sch.mod, target=target, params=params)
print(len(tasks))

In [ ]:

from mod_deploy import Module as ModuleAll
params_all = {}
tasks_all = auto_scheduler.extract_tasks(ModuleAll, params_all, target=target)
print(len(tasks_all))

In [ ]:
import numpy as np
log_file = "tune.json"
def _detect_local_cuda():
    dev = tvm.cuda()
    if not dev.exist:
        return None
    return tvm.target.Target(
        {
            "kind": "cuda",
            "max_shared_memory_per_block": dev.max_shared_memory_per_block,
            "max_threads_per_block": dev.max_threads_per_block,
            "thread_warp_size": dev.warp_size,
            "registers_per_block": 65536,
            "arch": "sm_" + tvm.cuda().compute_version.replace(".", ""),
        }
    )
# target = tvm.target.Target("cuda", host="llvm")
target = _detect_local_cuda()

print(target)
# 定义计算任务
dev = tvm.cuda(0)

num_flop = 1228406784
W_np = np.random.uniform(size=(512, vocab_size)).astype("uint32")
S_np = np.random.uniform(size=(128, vocab_size)).astype("float16")
Input_np = np.random.uniform(size=(1, 1, 4096)).astype("float16")
# Output_np = np.random.uniform(size=(1, 1, 4096)).astype("float16")
W_nd = tvm.nd.array(W_np, dev)
S_nd = tvm.nd.array(S_np, dev)
Input_nd = tvm.nd.array(Input_np, dev)
Output_nd = tvm.nd.array(np.zeros((1, 1, vocab_size), dtype="float32"), dev)
sch = tvm.tir.Schedule(ModuleSrc)
new_mod = sch.mod


In [ ]:
# task = auto_scheduler.SearchTask(func=sch.mod['fused_fused_decode11_fused_matmul5_cast2'], args=sch.mod['fused_fused_decode11_fused_matmul5_cast2'].params, target=target)

# tune_option = auto_scheduler.TuningOptions(
#     num_measure_trials=10,
#     measure_callbacks=[auto_scheduler.RecordToFile(log_file)],
#     verbose=2,
# )


database = ms.tune_tir(
    mod=new_mod,
    target=target,
    max_trials_global=64,
    num_trials_per_iter=64,
    work_dir="./tune_45593_1",
    cost_model="xgb"
)
print(len(database))
sch1 = ms.tir_integration.compile_tir(database, new_mod, target)
print(type(sch1))

In [ ]:
# print(sch1.trace)
# print(sch1.mod.script())
rt_mod = tvm.build(sch1.mod, target="cuda")

evaluator = rt_mod.time_evaluator("main", dev, number=100)

print("evaluator GEMV-Blocking: %f GFLOPS" % (1228406784 / evaluator(W_nd, S_nd, Input_nd, Output_nd).mean / 1e9))




In [ ]:

record_database = ms.Database.create(kind='json', work_dir='./tune_45593_1')


In [ ]:
record_sch = ms.tir_integration.compile_tir(record_database, new_mod, target)

record_rt_mod = tvm.build(record_sch.mod, target="cuda")

record_evaluator = record_rt_mod.time_evaluator("main", dev, number=20)

print("evaluator GEMV-Blocking: %f GFLOPS" % (num_flop / record_evaluator(W_nd, S_nd, Input_nd, Output_nd).mean / 1e9))
print(record_sch.trace)
print(record_sch.mod.script())

In [ ]:
from typing import TYPE_CHECKING, Dict, List, Optional, Union, Callable
from tvm import runtime
if TYPE_CHECKING:
    import numpy as np  # type: ignore
    from tvm.ir import IRModule
    from tvm.meta_schedule.runner import EvaluatorConfig, RPCConfig
    from tvm.runtime import Device, Module, NDArray
    from tvm.target import Target
    from tvm.runtime.vm import Executable


def f_measurement(
    rt_mod: runtime.Module, device: runtime.ndarray.Device, input_data: Dict[str, runtime.NDArray]
):
    vm = relax.VirtualMachine(rt_mod, device=device)
    vm.save_function("main", "measure_func", **input_data, include_return=False)
    evaluator = vm.time_evaluator(
        func_name="measure_func",
        dev=device,
        repeat=100,
        number=1,
        min_repeat_ms=500,
    )
    return evaluator()

def run_module_via_rpc(
    rpc_config: "RPCConfig",
    lib: Union["Module", "Executable"],
    dev_type: str,
    args: Union[Dict[int, "np.ndarray"], Dict[str, "np.ndarray"]],
    continuation: Callable,
    backend: Optional[str] = "graph",
):
    """Execute a tvm.runtime.Module on RPC remote"""
    # pylint: disable=import-outside-toplevel
    import os
    import tempfile

    from tvm.contrib.tar import tar
    from tvm.runtime import ndarray

    # pylint: enable=import-outside-toplevel

    with tempfile.TemporaryDirectory() as tmp_dir:
        # filename = os.path.join(tmp_dir, "tvm_tmp_mod." + tar.output_format)
        filename = os.path.join(tmp_dir, "tvm_tmp_mod." + "so")
        if backend == "vm":
            code, lib = lib.save(filename, fmt="so")
        from tvm.contrib import ndk
        lib.export_library(filename, ndk.create_shared)
        session = rpc_config.connect_server()
        print(type(session._sess))
        session.upload(filename)
        _, filename = os.path.split(filename)
        rt_mod = session.load_module(filename)
        
        if backend == "vm":
            rt_mod = session.get_function("runtime.Load_Executable")(code, rt_mod)
            # rt_mod = session.get_function("runtime.module.loadfile_relax.Executable")(filename)
        dev = session.device(dev_type=dev_type, dev_id=0)
        # print(dev)
        # create the remote runtime module
        print(rt_mod)
        print(rt_mod['main'])
        from tvm.contrib import graph_executor as runtime
        module = runtime.GraphModule(rt_mod["main"](dev))
        print(module)
        for k, v in args.items():
            module.set_input(k, tvm.nd.array(v))
        return module.run()
        # nd_args = {k: ndarray.array(v, dev) for k, v in args.items()}
        nd_args = {k: ndarray.empty(v.shape, v.dtype, dev) for k, v in args.items()}
        return continuation(rt_mod, dev, nd_args)